# Energy Consumption

IMPORT ALL REQUIRED LIBRARIES

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler,OrdinalEncoder,OneHotEncoder
from sklearn import feature_selection
from imblearn.over_sampling import RandomOverSampler
from sklearn.pipeline import Pipeline

import warnings
warnings.filterwarnings("ignore")

import template

In [ ]:
current_directory = os.getcwd()
data_dir = os.path.join(current_directory,"Datasets","Energy_consumption.csv")
data = pd.read_csv(data_dir)

In [ ]:
data.head()

In [ ]:
import template

In [ ]:
import pstats

In [ ]:
with cProfile.Profile() as pr:
    #data_val.missing_v(col_ty)
    data_val.missing_count(col_ty)

stats = pstats.Stats(pr)
stats.sort_stats(pstats.SortKey.TIME)
#stats.print_stats()
stats.dump_stats(filename="needs_profile.prof")


In [ ]:
data_val = template.data_val(data)

In [ ]:
col_ty = data_val.get_column_type()
print(col_ty)

In [ ]:
data.count()

In [ ]:
def unique_count(df,column_types):
    unique= {}
    unique_counts = {}
    for column in df.columns:
        if column_types[column] == "categorical" and column != "Timestamp":
            unique[column] = pd.unique(df[column])
            unique_counts[column] = len(unique[column])
    uniques = { "unique_values" : unique,"unique_counts" : unique_counts}
    return uniques

In [ ]:
print(unique_count(data,col_ty))

In [ ]:
def zero_count(df):
    zero_counts = {}
    for column in df.columns:
        zero_counts[column] = column.count(axis=1)
        
    print(zero_counts)
zero_count(data)


In [ ]:
data_val.missing_count(col_ty)

In [ ]:
num_info = data.info()
print("hi",num_info)

In [ ]:
cProfile.run("data.info()")

In [ ]:
data.head()

In [ ]:
data.metadata()

In [ ]:
data["Timestamp"] = pd.to_datetime(data["Timestamp"])

data.set_index("Timestamp",inplace=True)

In [ ]:
data.head()

In [ ]:
sns.set_theme()
sns.relplot(data=data,x=data["Temperature"],y=data["EnergyConsumption"])

In [ ]:
x = data["Temperature"]
y = data["EnergyConsumption"]

In [ ]:
x_train1,x_test1,y_train1,y_test1 = train_test_split(x,y,test_size=0.2,random_state=42)
x_train = x_train1.values.reshape(-1,1)
y_train = y_train1.values.reshape(-1,1)
x_test = x_test1.values.reshape(-1,1)
y_test = y_test1.values.reshape(-1,1)


In [ ]:
regr = linear_model.LinearRegression()
regr.fit(x_train, y_train)
y_pred = regr.predict(x_test)
print("Coefficients of regression : ",regr.coef_)
print("mean_squared_error : %.2f"% mean_squared_error(y_test, y_pred))
print("Coefficients of determination :%.2f "% r2_score(y_test, y_pred))

In [ ]:
z = {"Temperature": x_train1, "EnergyConsumpition": y_train1, "x_test": x_test1, "y_test":y_test1}
z= pd.DataFrame(z)

In [ ]:
m = regr.coef_.reshape(-1)
b = regr.intercept_.reshape(-1)
sns.relplot(data=z,x="Temperature",y="EnergyConsumpition")
x_val = np.linspace(z["Temperature"].min(),z["Temperature"].max(),100)
y_val = m*x_val + b
plt.plot(x_val,y_val,color = "blue")

Overall, an R^2 value of 0.45 suggests that the regression model has moderate predictive power, explaining a significant portion but not all of the variance in the dependent variable. However, a mean squared error of 35.81 indicates that there is still room for improvement in reducing prediction errors. So, while the model is performing decently, there is still scope for enhancement.

So now we will use another algorithm
based on type of data and their relationship we can use different algorithms such as :
1. Multiple linear regression
2. Decision Tree Regressor
3. Random Forest
4. Gradient Boosting Regressor
5. Support Vector regressor
6. Neural Network

but from data we can see more categorical data is available so i think that decision tree and random forest can be greate algorithm which is suitable for this type of data

In [ ]:
x1 = data.iloc[:,:9]
y1 = data["EnergyConsumption"]

In [ ]:
x1["SquareFootage"] = x1["SquareFootage"]/100

In [ ]:
x1_train,x1_test,y1_train,y1_test = train_test_split(x1,y1,test_size=0.3,random_state=42)

In [ ]:
y1_train.head()

In [ ]:
def pipeline_transform(xa):
    x2 = xa["HVACUsage"].values.reshape(-1,1)
    x3 = xa["LightingUsage"].values.reshape(-1,1)
    x4 = xa["DayOfWeek"].values.reshape(-1,1)
    x5 = xa["Holiday"].values.reshape(-1,1)
    enc = preprocessing.OneHotEncoder(drop='if_binary')
    enc.fit(x2)
    xa["HVACUsage"] = enc.transform(x2).toarray()
    enc = preprocessing.OneHotEncoder(drop='if_binary')
    enc.fit(x3)
    xa["LightingUsage"] = enc.transform(x3).toarray()
    enc = preprocessing.OneHotEncoder()
    enc.fit(x4)
    xa["DayOfWeek"] = enc.transform(x4).toarray()
    enc = preprocessing.OneHotEncoder(drop='if_binary')
    enc.fit(x5)
    xa["Holiday"] = enc.transform(x5).toarray()
    return xa





In [ ]:
x1.head()

In [ ]:
x1_test.head()

In [ ]:
x1_train = pipeline_transform(x1_train)
x1_test = pipeline_transform(x1_test)

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor
reg1 = HistGradientBoostingRegressor().fit(x1_train, y1_train)
print(reg1.score(x1_test, y1_test),reg1.score(x1_train, y1_train))

In [ ]:
from sklearn import svm
reg2  = svm.SVR()
reg2.fit(x1_train, y1_train)
print(reg2.score(x1_test, y1_test),reg2.score(x1_train, y1_train))

In [ ]:
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
reg3 = make_pipeline(StandardScaler(),SGDRegressor(max_iter=1000, tol=1e-3))
reg3.fit(x1_train, y1_train)
print(reg3.score(x1_test, y1_test),reg3.score(x1_train, y1_train))


In [ ]:
from sklearn.ensemble import RandomForestRegressor
reg4 = RandomForestRegressor(max_depth=4,random_state=42)
reg4.fit(x1_train, y1_train)
print(reg4.score(x1_test, y1_test),reg4.score(x1_train, y1_train))

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
reg5 = AdaBoostRegressor(random_state=42,n_estimators=100)
reg5.fit(x1_train, y1_train)
print(reg5.score(x1_test, y1_test),reg5.score(x1_train, y1_train))


In [ ]:
from sklearn.ensemble import BaggingRegressor
from sklearn.svm import SVR

reg6 = BaggingRegressor(estimator=SGDRegressor(max_iter=1000, tol=1e-3),random_state=42,n_estimators=100)

reg6.fit(x1_train, y1_train)

print(reg6.score(x1_test, y1_test),reg6.score(x1_train, y1_train))

In [ ]:
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import train_test_split

# Assuming x1_train, x1_test, y1_train, y1_test are defined
reg6 = BaggingRegressor(estimator=SGDRegressor(max_iter=1000, tol=1e-3), random_state=42, n_estimators=100)

reg6.fit(x1_train, y1_train)

print(reg6.score(x1_test, y1_test), reg6.score(x1_train, y1_train))


In [ ]:
from sklearn.ensemble import VotingRegressor
ereg = VotingRegressor(estimators=[('Hgb', reg1), ('svr', reg2), ('sgd', reg3),('rd',reg4),('ada',reg5)])
ereg.fit(x1_train, y1_train)
print(ereg.score(x1_test, y1_test),ereg.score(x1_train, y1_train))